# 27.基于MindSpore实现强化学习示例
   
   本小节介绍基于mindspore实现强化学习示例。

# 1.实验目的
- 理解强化学习（Reinforcement Learning）的相关概念。
- 理解智能体通过与环境的交互进行学习的流程。

# 2. 知识点介绍


## 2.1 强化学习（Reinforcement Learning）

- 强化学习（Reinforcement Learning）是机器学习的一个分支，旨在让智能体（agent）通过与环境的交互来学习最优行为策略，以最大化累积奖励或获得特定目标。


### 2.1.1 强化学习的基本元素包括：

- 1.环境（Environment）：智能体与其交互的外部环境，可以是真实世界或模拟环境。
- 2.状态（State）：环境的某个特定时刻的观测值，用于描述环境的特征。
- 3.动作（Action）：智能体在某个状态下可以执行的操作或决策。
- 4.奖励（Reward）：在特定状态下，环境根据智能体的动作给予的反馈信号。
- 5.策略（Policy）：智能体的行为策略，决定在给定状态下采取哪个动作。
- 6.值函数（Value Function）：评估状态或状态-动作对的优劣程度，用于指导决策。
- 7.学习算法（Learning Algorithm）：根据智能体与环境的交互经验，更新策略或值函数的算法。

### 2.1.2 强化学习的目标：
- 通过学习最优策略来最大化累积奖励。为了达到这个目标，智能体通过与环境的交互，观察当前状态，选择动作，接收奖励，并根据奖励信号来调整策略或值函数。强化学习算法可以分为基于值函数的方法（如Q-learning、DQN）和基于策略的方法（如Policy Gradient、Actor-Critic）等。

### 2.1.3 应用
- 强化学习在许多领域都有应用，例如机器人控制、游戏玩法、自动驾驶、金融交易等。它的独特之处在于智能体通过与环境的交互进行学习，而无需依赖标注的数据集，因此适用于很多现实世界的场景，可以在复杂、未知和动态的环境中进行决策和学习。

## 2.2 DQN 算法

- DQN（Deep Q-Network）是一种用于解决强化学习问题的深度学习算法。它结合了Q-learning和神经网络，能够学习并估计状态-动作对的Q值函数，通过使用神经网络来逼近Q值函数，实现了对高维状态空间的学习和泛化能力。


$$
\begin{split}
&Q(s_t,a_t) = Q(s_t,a_t) + \eta\cdot (R_{t+1}+\gamma \max_a Q(s_{t+1},a)-Q(s_t,a_t))\\
&Q(s_t,a_t) = R_{t+1}+\gamma\cdot \max_aQ(s_{t+1},a)
\end{split}
$$

- iteration algorithm
- temporal difference error（TD）: $R_{t+1}+\gamma\cdot \max_aQ(s_t,a) - Q(s_t,a_t)$
- learning objective：
    - MES（square loss）: $E(s_t,a_t)=\left(R_{t+1}+\gamma\max_aQ(s_{t+1},a)-Q(s_t,a_t)\right)^2$

### 2.2.1 核心知识
- experience replay（经验回放）
    - 不像 q-table 的 q-learning，每一步都学习（update）该步的内容（experience）
    - 对于 q-table 而言，每一步（step）都学习该步的内容，神经网络连续地学习时间上相关性高的内容（事实上，时间 $t$ 的学习内容，和时间         - $t+1$ 的学习内容非常相似，这样的话，收敛就会很慢；
    - 而是将每一步（step）的内容存储在经验池（experience pool）并随机从经验池中提取内容（replay，回放）让NN学习；
    - 也是一种批次化（batch），使用经验池中的多个步骤的经验。

###  2.2.2 DQN算法的基本步骤：

- 1.初始化Q网络，目标Q网络和经验回放缓冲区。
- 2.对于每个回合（episode）循环：
     - 重置环境并获取初始状态。
     - 对于每一步（step）循环：根据当前状态从Q网络中选择一个动作（通常使用ε-greedy策略，其中ε是一个随时间递减的参数）。
     - 执行选定的动作，观察下一个状态和奖励。
     - 将转移（当前状态，动作，下一个状态，奖励）存储在经验回放缓冲区中。
     - 从经验回放缓冲区中随机采样一批转移。
     - 计算目标Q值：对于每个样本转移，计算目标Q值作为
      - target_Q = reward + discount_factor * max(Q(next_state, next_action))
     - 其中 next_action 是从目标Q网络中选择的下一个动作。
     - 通过最小化目标Q值和当前Q值的均方误差来更新Q网络的参数。
     - 定期更新目标Q网络的参数，例如每隔一定的步数。
     - 更新当前状态为下一个状态。
     - 如果达到终止条件（例如，达到最大步数或解决了环境），则跳出循环。
- 3.返回训练好的Q网络。

## 2.3 gym平台介绍
本实验借助了gym平台的环境，该平台由 openai 公司开发，且提供了一整套与平台中虚拟环境进行交互的 api接口，gym 的推出为强化学习算法的研究提供了更好地基准测试平台，同时将各类 环境标准化，使研究员可以专注于算法研究而无需花过多的时间在环境的模拟上。gym 提供一个 step 函数供智能体与环境进行交互，其参数为动作，主要返回值及含义分别为：
- state：表示智能体所处环境的当前状态，代表着智能体的观察值即状态。
- reward：表示智能体采取操作后从环境中获得的奖励，其类型可能是整数、小数等，但是具体的规模和类型与具体的规模和类型与环境有关，但是智能体的总目标仍然是获取最大的奖励值。
- done: 大多数任务都属于阶段性任务，当到达一定条件的时候表示任务已经结束，比如五子棋游戏中的一方五子相连，机器人在路面上摔倒，或者在规定的步数以内没有完成任务，则都代表任务结束。所以 done 是一个判断条件，类型为布尔值，代表当前任务是否结束，如果结束则可以选择使用 reset 函数重置当前任务。

# 3. 实验环境
在动手进行实践之前，需要注意以下几点：
* 确保实验环境正确安装，包括安装MindSpore。安装过程：首先登录[MindSpore官网安装页面](https://www.mindspore.cn/install)，根据安装指南下载安装包及查询相关文档。同时，官网环境安装也可以按下表说明找到对应环境搭建文档链接，根据环境搭建手册配置对应的实验环境。
* 推荐使用交互式的计算环境Jupyter Notebook，其交互性强，易于可视化，适合频繁修改的数据分析实验环境。
* 实验也可以在华为云一站式的AI开发平台ModelArts上完成。
* 推荐实验环境：MindSpore版本=1.8；Python环境=3.7


|  硬件平台 |  操作系统  | 软件环境 | 开发环境 | 环境搭建链接 |
| :-----:| :----: | :----: |:----:   |:----:   |
| CPU | Windows-x64 | MindSpore1.8 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.1节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| GPU CUDA 10.1|Linux-x86_64| MindSpore1.8 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.2节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| Ascend 910  | Linux-x86_64| MindSpore1.8 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第四章](./MindSpore环境搭建实验手册.docx)|

# 4. 数据处理

## 4.1 实验准备

In [1]:
import numpy as np
import gym
import mindspore as ms
import mindspore.nn as nn
# 常见算子操作
from mindspore import  ops

## 4.2 数据加载

In [2]:
# Hyper Parameters
BATCH_SIZE = 8
LR = 0.01                   # learning rate
EPSILON = 0.9               # greedy policy
GAMMA = 0.9                 # reward discount
TARGET_REPLACE_ITER = 100   # target update frequency
MEMORY_CAPACITY = 2000
# 加载车杆模型
env = gym.make('CartPole-v1')
env = env.unwrapped
# 获取动作数
N_ACTIONS = env.action_space.n
# 获取状态数
N_STATES = env.observation_space.shape[0]
ENV_A_SHAPE = 0 if isinstance(env.action_space.sample(), int) else env.action_space.sample().shape     # to confirm the shape

# 5.算法实现
- 使用MindSpore实现DQN算法并进行训练。

## 5.1 导入Python库并配置运行信息

In [6]:
# 常见算子操作
from mindspore.ops import function as F
# 引入张量模块
from mindspore import Tensor

## 5.2 定义神经网络模型 Net

定义了一个神经网络模型 Net，使用了 mindspore 框架的 nn 模块。

In [4]:
class Net(nn.Cell):
    def __init__(self, ):
        super(Net, self).__init__()
        # 一个全连接层
        self.fc1 = nn.Dense(N_STATES, 20)
        # 第二个全连接层
        self.fc2 = nn.Dense(20, 50)
        # 第s三个全连接层
        self.fc3 = nn.Dense(50, N_ACTIONS)
        # 激活函数
        self.relu = nn.ReLU()
        
    def construct(self, x):
        # 全连接层
        x = self.fc1(x)
        # 全连接层
        x = self.fc2(x)
        # 激活层
        x = self.relu(x)
        # 全连接层
        x = self.fc3(x)
        # 输出
        actions_value = self.relu(x)
        # 返回输出值
        return actions_value

这个网络模型具有三个全连接层和一个激活函数层，可以接受输入 x 并输出 actions_value。在神经网络中，这个模型的作用是将输入映射到输出，用于近似 Q-值函数的估计。

## 5.3 构建DQN

实现DQN（Deep Q-Network）算法的类 DQN，包括网络的初始化、动作选择、记忆存储和学习过程。

### 5.3.1 初始化：

- 初始化DQN智能体。创建两个Net类的实例（eval_net和target_net）。
- learn_step_counter用于跟踪目标网络更新的步数。
- memory_counter用于跟踪存储在内存中的转换数。
- memory数组用于存储转换（状态、动作、奖励、下一个状态）。
- optimizer是Adam优化器的实例，用于更新神经网络参数。
- loss_func是均方误差（MSE）损失函数，用于计算损失。

In [2]:
class DQN(object):
    def __init__(self):
        self.eval_net, self.target_net = Net(), Net()
        # 更新目标
        self.learn_step_counter = 0                                     # for target updating
        # 存储记忆
        self.memory_counter = 0                                         # for storing memory
        # 记忆初始化
        self.memory = np.zeros((MEMORY_CAPACITY, N_STATES * 2 + 2))     # initialize memory
        # Adaptive Moment Estimation 动态调整参数学习率
        self.optimizer = nn.Adam(self.eval_net.trainable_params(), learning_rate=LR)
       # MSE损失函数
        self.loss_func = nn.MSELoss()

### 5.3.2 选择动作：
- choose_action方法以状态x作为输入，并根据ε-贪心策略选择动作。
- 如果随机生成的数小于探索率（EPSILON），则从评估网络（eval_net）中选择具有最高Q值的动作。
- 否则，选择一个随机动作。

In [3]:
    def choose_action(self, x):
        # 输入样本
        x = Tensor([x])
        # greedy贪心
        if np.random.uniform() < EPSILON:   
            # 获取所有动作值
            actions_value = self.eval_net(x)
            # 获取最优动作
            action = np.array(ops.max(actions_value), dtype=int)
            # 调整维度
            action = action[0] if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)  # return the argmax index
        else:   
            # 随机获取动作值
            action = np.random.randint(0, N_ACTIONS)
            # 调整维度
            action = action if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)
        action = np.array(action, dtype=int)
        # 返回动作值
        return action

### 5.3.3 存储转换：

- store_transition方法将转换（状态、动作、奖励、下一个状态）存储在内存中。
- 它将转换的各个组件连接成一个数组，并将其添加到内存的当前内存计数器索引处。
- 内存计数器递增，以跟踪存储的转换数。

In [4]:
    def store_transition(self, s, a, r, s_):
        transition = np.hstack((s, [a, r], s_))
        # 更新记忆内容
        index = self.memory_counter % MEMORY_CAPACITY
        self.memory[index, :] = transition
        self.memory_counter += 1

### 5.3.4 学习：

- learn方法通过训练评估网络（eval_net）来更新DQN。
- 首先，它根据learn_step_counter判断是否是更新目标网络（target_net）的时机。
- 然后，从内存中采样一批转换，并将其分离为各个组件（状态、动作、奖励、下一个状态）。
- 使用评估网络计算选定动作的当前Q值。
- 通过考虑奖励和目标网络中下一个状态的最大Q值来计算目标Q值。
- 损失函数用于计算当前Q值和目标Q值之间的损失。
- 优化器用于根据损失更新评估网络的参数。

In [5]:
    def learn(self):
        # 目标参数更新
        if self.learn_step_counter % TARGET_REPLACE_ITER == 0:
            # 加载网络参数
            ms.load_param_into_net(self.target_net, self.eval_net.parameters_dict())
        self.learn_step_counter += 1
        # sample batch transitions
        sample_index = np.random.choice(MEMORY_CAPACITY, BATCH_SIZE)
        b_memory = self.memory[sample_index, :]
        b_s = Tensor([b_memory[:, :N_STATES]])
        b_a = Tensor([b_memory[:, N_STATES:N_STATES+1].astype(int)])
        b_r = Tensor(b_memory[:, N_STATES+1:N_STATES+2])
        b_s_ = Tensor([b_memory[:, -N_STATES:]])
        # 计算现实Q值
        q_eval = self.eval_net(b_s).gather_elements(2, b_a)  # shape (batch, 1
        # 从计算图中分离，阻止反向传播
        q_next = ops.diag(self.target_net(b_s_))
        # 计算目标Q值
        q_target = b_r + GAMMA * q_next[0][0][0].max(2).view((BATCH_SIZE, 1))   # shape (batch, 1)
        # 损失函数
        loss_net = nn.WithLossCell(self.eval_net, loss_fn = self.loss_func(q_eval, q_target))
        # 梯度更新
        train_net = nn.TrainOneStepCell(loss_net, self.optimizer)

In [6]:
dqn = DQN()

# 6.模型训练

In [8]:
print('开始训练...')
for i_episode in range(300):
    # 重置环境
    s = env.reset()
    # 总奖励
    ep_r = 0
    while True:
        # env.render()
        a = dqn.choose_action(s)
        # 获取训练过程相关参数
        s_, r, done, info = env.step(a)
        # 修改奖励
        x, x_dot, theta, theta_dot = s_
        # 计算奖励值
        r1 = (env.x_threshold - abs(x)) / env.x_threshold - 0.8
        r2 = (env.theta_threshold_radians - abs(theta)) / env.theta_threshold_radians - 0.5
        r = r1 + r2
        # 更新记忆
        dqn.store_transition(s, a, r, s_)
        # 计算总奖励
        ep_r += r
        if dqn.memory_counter > MEMORY_CAPACITY:
            # 从记忆中学习
            dqn.learn()
            if done:
                print('Epoch: ', i_episode, '| Epoch_reward: ', round(ep_r, 2))

        if done:
            break
        s = s_


开始训练...
Epoch:  194 | Epoch_reward:  1.39
Epoch:  195 | Epoch_reward:  3.15
Epoch:  196 | Epoch_reward:  2.67
Epoch:  197 | Epoch_reward:  2.46
Epoch:  198 | Epoch_reward:  2.84
Epoch:  199 | Epoch_reward:  1.62
Epoch:  200 | Epoch_reward:  2.59
Epoch:  201 | Epoch_reward:  4.15
Epoch:  202 | Epoch_reward:  2.66
Epoch:  203 | Epoch_reward:  2.27
Epoch:  204 | Epoch_reward:  0.77
Epoch:  205 | Epoch_reward:  1.58
Epoch:  206 | Epoch_reward:  3.05
Epoch:  207 | Epoch_reward:  1.11
Epoch:  208 | Epoch_reward:  2.52
Epoch:  209 | Epoch_reward:  2.5
Epoch:  210 | Epoch_reward:  1.71
Epoch:  211 | Epoch_reward:  2.42
Epoch:  212 | Epoch_reward:  1.51
Epoch:  213 | Epoch_reward:  2.91
Epoch:  214 | Epoch_reward:  2.32
Epoch:  215 | Epoch_reward:  1.89
Epoch:  216 | Epoch_reward:  1.79
Epoch:  217 | Epoch_reward:  2.19
Epoch:  218 | Epoch_reward:  1.35
Epoch:  219 | Epoch_reward:  1.26
Epoch:  220 | Epoch_reward:  2.25
Epoch:  221 | Epoch_reward:  2.66
Epoch:  222 | Epoch_reward:  2.35
Epoch: 